In [1]:
#r "C:\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


In [1]:
var tempDB = OpenOrCreateDatabase("C:\\tmp\\Newton");

In [1]:
databases;

In [1]:
ExecutionQueues;

In [1]:
var mainDB = tempDB;

In [1]:
var mainQueue = ExecutionQueues[0];

In [1]:
WorkflowMgm.Init("NonlinearSolverBenchmark");

In [1]:
WorkflowMgm.SetEqualityBasedSessionJobControllCorrelation();

In [1]:
using BoSSS.Application.IBM_Solver;

In [1]:
// --------------
// specify meshes 
// --------------

In [1]:
// grid creation loop:

In [1]:
IGridInfo[] grids = new IGridInfo[1];
for(int i = 0; i < grids.Length; i++) {
    var gDb = mainDB.Grids.SingleOrDefault(gi => gi.ID == new Guid("3572ea5c-3090-4c26-a479-260809a7c40c"));
    if(gDb != null) {
        // grid already created
        Console.WriteLine("found " + gDb.ToString());
        grids[i] = gDb;
        continue;
    }
    
    int nx = 21*5*(i+1);
    int ny = 21*(i+1);
    int nz = ny;
    double[] xNodes = GenericBlas.Linspace(-5,+5, nx + 1);   
    double[] yNodes = GenericBlas.Linspace(-1,+1, ny + 1);
    double[] zNodes = GenericBlas.Linspace(-1,+1, nz + 1);
    
    
    Console.WriteLine("Creating grid with " + (nx*ny*nz) + " cells ...");
    
    var grd = Grid3D.Cartesian3DGrid(xNodes, yNodes, zNodes);
    
    
    grd.DefineEdgeTags(delegate(double[] X) {
       double x = X[0];
       if(Math.Abs(x - (-5.0)) < 1e-8) {
           return "Velocity_Inlet_front";    
           
       } else if(Math.Abs(x - (+5.0)) < 1.0e-8) {
           return "Pressure_Outlet_back";    
       } else {
           return "walls";
       }
    });
    
    Console.WriteLine("done.");
    Console.WriteLine("Saving to database ...");
    mainDB.SaveGrid(ref grd);
    grids[i] = grd;
    Console.WriteLine("done.");
}

In [1]:
// -------------------------
// further parameter ranges
// -------------------------

In [1]:
var codes = new NonLinearSolverCode[]{NonLinearSolverCode.Picard, NonLinearSolverCode.Newton } ;

In [1]:
var ReynoldsS = new double[] { 1, 2, 10, 20, 100, 200.0 };

In [1]:
var DGdegree = new int[] { 1, 2, 3 };

In [1]:
// -----------------------
// specify control objects
// -----------------------

In [1]:
//grids[0].Describe();

In [1]:
var controls = new List<IBM_Control>();

In [1]:
controls.Clear();
foreach(var g in grids) {
foreach(var p in DGdegree) {
foreach(var Rey in ReynoldsS) {
    var c1 = new IBM_Control();
    c1.SetDGdegree(p);
    c1.SetGrid(g);
    c1.SetDatabase(mainDB);
    c1.SessionName = string.Format("IBMsphere-Rey{0}-J{0}p{1}", Rey, g.NumberOfCells, p);

    c1.InitialValues.Add("Phi", 
       new Formula("X => -(X[0]).Pow2() + -(X[1]).Pow2() + -(X[2]).Pow2() + 0.5"));
    c1.AddBoundaryValue("Velocity_Inlet_front", "VelocityX", 
       new Formula("X => 1.0"));
    c1.AddBoundaryValue("walls", "VelocityX", 
      new Formula("X => 1.0"));

    c1.PhysicalParameters.rho_A = 1.0;
    c1.PhysicalParameters.mu_A  = 1.0/Rey;
    c1.TimesteppingMode         = AppControl._TimesteppingMode.Steady;
}
}
}

In [1]:
foreach(var c1 in controls) {
    var j1 = c1.CreateJob();
    j1.Activate(mainQueue);
}

In [1]:
j1.RetryCount = 1;

In [1]:
j1.Activate(mainQueue);

In [1]:
j1.ShowOutput();

In [1]:
j1.Status;

In [1]:
j1.Stdout;

In [1]:
j1.Stderr;

In [1]:
j1.BatchProcessorIdentifierToken;

In [1]:
j1.LatestSession.DeployPath;

In [1]:
j1.DeploymentDirectory;

In [1]:
//j1.LatestSession.OpenSessionDirectory();

In [1]:
var rl = j1.LatestSession.Residuals();

In [1]:
rl;

In [1]:
LastError;

In [1]:
rl.GetType();

In [1]:
rl.Values.Keys

In [1]:
static class MyUtils {

    public static int[] Indices4Ts(ResidualLog log, int Timestep) {
        var r         = new List<int>();
        var timesteps = log.Values["#Time"];
        for(int a = 0; a < timesteps.Count; a++) {
            if(timesteps[a] == Timestep) {
                r.Add(a);
            }
        }
        return r.ToArray();
    }
    
    static public double[] TotResidual(ResidualLog log, int Timestep) {
        string[] Cols = log.Values.Keys.Where(name => !name.StartsWith("#")).ToArray();
        int[] lines = Indices4Ts(log, Timestep);
        
        double[] ret = new double[lines.Length];
        foreach(var col in Cols) {
            var colVals = log.Values[col];    
            for(int i = 0; i < lines.Length; i++) {
                ret[i] += colVals[lines[i]].Pow2();    
            }
        }
        
        for(int i = 0; i < lines.Length; i++) {
            ret[i] = Math.Sqrt(ret[i]);
        }
        
        return ret;
    }
}

In [1]:
MyUtils.Indices4Ts(rl, 2);

In [1]:
MyUtils.TotResidual(rl, 1);